In [192]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import math

In [193]:
def checkSpecial(url):
    """Returns number of special characters in string"""
    regex = re.compile('[@_!#$%^&*()<>?|}{~]')
    return len([c for c in url if regex.search(c)])

def getNums(url):
    """Returns number of digits in string"""
    return len([c for c in url if c.isdigit()])

def entropy(url):
    """Returns entropy of string"""
    s = url.strip()
    prob = [float(s.count(c)) / len(s) for c in dict.fromkeys(list(s))]
    ent = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
    return ent

def numSubDomains(url):
    """Returns number of subdomains in the given URL"""
    subdomains = url.split('http')[-1].split('//')[-1].split('/')
    return len(subdomains)-1

def feature_transform(df):
    """Featurizes the URL string into the data frame"""
    df.insert(2, 'len_url', [len(url) for url in df['URL']])
    df.insert(2, 'numerical', [getNums(url) for url in df['URL']])
    df.insert(2, 'special', [checkSpecial(url) for url in df['URL']])
    df.insert(2, 'hasPercent', [1 if ('%' in url) else 0 for url in df['URL']])
    df.insert(2, 'entropy', [entropy(url) for url in df['URL']])
    df.insert(2, 'numSD', [numSubDomains(url) for url in df['URL']])
    del df['URL']

In [194]:
df = pd.read_csv('Phishing_Dataset.csv')
feature_transform(df)

y = df.pop('Label')

x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

def norm(x):
  return (x - x.mean())/x.std()

normed_train = norm(x_train)
normed_test = norm(x_test)

In [195]:
def build_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=[len(x_train.columns)]),
     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(25, activation='relu', kernel_initializer='he_normal'),
     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [196]:
model = build_model()
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_94 (Dense)             (None, 100)               1000      
_________________________________________________________________
dropout_23 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_95 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_96 (Dense)             (None, 25)                1275      
_________________________________________________________________
dropout_24 (Dropout)         (None, 25)                0         
_________________________________________________________________
dense_97 (Dense)             (None, 1)                 26        
Total params: 7,351
Trainable params: 7,351
Non-trainable params: 0
___________________________________________________

In [197]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=5)

history = model.fit(normed_train, y_train, epochs=150, batch_size=32, validation_split=0.2, callbacks=[callback])

Epoch 1/150
96/96 [==============================] - 0s 3ms/step - loss: 0.4251 - accuracy: 0.8147 - val_loss: 0.2749 - val_accuracy: 0.8958
Epoch 2/150
96/96 [==============================] - 0s 1ms/step - loss: 0.3071 - accuracy: 0.8824 - val_loss: 0.2604 - val_accuracy: 0.8971
Epoch 3/150
96/96 [==============================] - 0s 2ms/step - loss: 0.2920 - accuracy: 0.8854 - val_loss: 0.2517 - val_accuracy: 0.9010
Epoch 4/150
96/96 [==============================] - 0s 2ms/step - loss: 0.2773 - accuracy: 0.8951 - val_loss: 0.2437 - val_accuracy: 0.9049
Epoch 5/150
96/96 [==============================] - 0s 2ms/step - loss: 0.2758 - accuracy: 0.8922 - val_loss: 0.2437 - val_accuracy: 0.9036
Epoch 6/150
96/96 [==============================] - 0s 2ms/step - loss: 0.2609 - accuracy: 0.8942 - val_loss: 0.2415 - val_accuracy: 0.9115
Epoch 7/150
96/96 [==============================] - 0s 2ms/step - loss: 0.2636 - accuracy: 0.8981 - val_loss: 0.2308 - val_accuracy: 0.9102
Epoch 8/150
9

In [198]:
score = model.evaluate(normed_test, y_test)

# test_predictions = model.predict(normed_test).flatten()
# predictions = [int(round(value)) for value in test_predictions]
# print(predictions)

30/30 [==============================] - 0s 846us/step - loss: 0.2109 - accuracy: 0.9135
